# Only template

I will try streamlit later

In [ ]:
import streamlit as st
import shap
import lime
import lime.lime_tabular
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit.components.v1 as components

# ============ PLACEHOLDER: replace with your own model & data ============
# Example model, X, test sample
def load_model_and_data():
    from xgboost import XGBRegressor
    from sklearn.datasets import fetch_california_housing
    from sklearn.model_selection import train_test_split

    data = fetch_california_housing()
    X = pd.DataFrame(data.data, columns=data.feature_names)
    y = data.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBRegressor().fit(X_train, y_train)
    return model, X_test.reset_index(drop=True), y_test.reset_index(drop=True)

model, test_X, test_y = load_model_and_data()

# ======================== Streamlit UI =========================
st.title("🔍 SHAP vs LIME Explanation Dashboard")

sample_idx = st.slider("Select a sample index", 0, len(test_X)-1, 0)
method = st.selectbox("Choose explanation method", ["SHAP", "LIME"])

X_sample = test_X.iloc[[sample_idx]]

# ======================== SHAP =========================
@st.cache_resource
def get_shap_explainer():
    return shap.Explainer(model, test_X)

if method == "SHAP":
    st.subheader("SHAP Explanation")
    explainer = get_shap_explainer()
    shap_values = explainer(X_sample)

    fig = shap.plots._waterfall.waterfall_legacy(shap_values[0], show=False)
    st.pyplot(fig)

# ======================== LIME =========================
elif method == "LIME":
    st.subheader("LIME Explanation")
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=np.array(test_X),
        feature_names=test_X.columns,
        mode='regression',
        verbose=False
    )

    lime_exp = lime_explainer.explain_instance(
        data_row=np.array(X_sample.iloc[0]),
        predict_fn=model.predict
    )

    # Save to html and embed
    html = lime_exp.as_html()
    components.html(html, height=800, scrolling=True)

# ======================== Extra Info =========================
st.markdown("---")
st.markdown(f"**Predicted value**: `{model.predict(X_sample)[0]:.2f}`")
st.markdown(f"**True value**: `{test_y.iloc[sample_idx]:.2f}`")
